In [4]:
import tarfile
import gzip
import pandas as pd
import os
import yaml
import numpy as np
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 메타데이터 추출 함수
def extract_package_metadata(input_path="../data/package", 
                             output_path="../data/metadata/raw", 
                             output_filename="extracted_metadata.csv"):
    
    # 31개의 컬럼 정의
    columns = ['id', 'name', 'version', 'authors', 'email', 'summary', 'description', 'date',
               'files', 'test_files', 'autorequire', 'executables', 'require_paths',
               'dependencies', 'runtime_dependencies', 'development_dependencies', 'extensions', 'requirements',
               'homepage', 'metadata', 'licenses',
               'platform', 'required_ruby_version', 'required_rubygems_version', 'rubygems_version',
               'extra_rdoc_files', 'rdoc_options', 'specification_version',
               'cert_chain', 'signing_key', 'post_install_message']

    data = []

    for root, dirs, files in os.walk(input_path):
        for file in files:
            if file.endswith(".gem"):
                gem_file_path = os.path.join(root, file)
                try:
                    with tarfile.open(gem_file_path, 'r') as gem_file:
                        # metadata.gz 파일을 추출
                        metadata_gz = next((gem_file.extractfile(member) for member in gem_file.getmembers() if member.name.endswith("metadata.gz")), None)
                        if metadata_gz is None:
                            logging.warning(f"metadata.gz not found in {gem_file_path}")
                            continue

                        # metadata.gz 파일을 gzip 해제
                        with gzip.GzipFile(fileobj=metadata_gz) as f:
                            metadata_content = f.read()

                        # YAML 파싱 전에 특수 태그 제거 (replace를 한 번에 처리)
                        tags_to_remove = ["!ruby/object:Gem::Specification", "!ruby/object:Gem::Version", 
                                          "!ruby/object:Gem::Dependency", "!ruby/object:Gem::Requirement"]
                        metadata_text = metadata_content.decode("utf-8")
                        for tag in tags_to_remove:
                            metadata_text = metadata_text.replace(tag, "")

                        # YAML 파싱
                        metadata = yaml.safe_load(metadata_text)

                        # 각 특성 추출 및 null 값 처리
                        row = []
                        name = metadata.get('name', '')
                        version = metadata.get('version', '')
                        # id 컬럼 생성
                        id_value = f"{name}-{version}" if name and version else np.nan
                        row.append(id_value)  # id 추가
                        
                        for col in columns[1:]:  # 이미 id는 추가했으므로 나머지 컬럼만 처리
                            value = metadata.get(col, np.nan)  # 기본적으로 np.nan으로 처리
                            if isinstance(value, list) and not value:  # 빈 리스트 처리
                                value = np.nan
                            if isinstance(value, str) and (not value.strip() or value in [' ', '']):  # 빈 문자열, 띄어쓰기, 공백 처리
                                value = np.nan
                            if value is None:  # None 값도 np.nan으로 처리
                                value = np.nan
                            row.append(value)

                        # dependencies를 runtime과 development로 분리하여 추출
                        dependencies = metadata.get('dependencies', np.nan)
                        runtime_dependencies = []
                        development_dependencies = []

                        if dependencies is not np.nan:
                            for dep in dependencies:
                                dep_type = dep.get('type', '')
                                if dep_type == ':runtime':
                                    runtime_dependencies.append(dep)
                                elif dep_type == ':development':
                                    development_dependencies.append(dep)

                            if not runtime_dependencies:
                                runtime_dependencies = np.nan
                            if not development_dependencies:
                                development_dependencies = np.nan
                        else:
                            runtime_dependencies = np.nan
                            development_dependencies = np.nan

                        # dependencies를 원본 그대로 유지하고, 분류된 것을 각각 추가
                        row[columns.index('dependencies')] = dependencies
                        row[columns.index('runtime_dependencies')] = runtime_dependencies
                        row[columns.index('development_dependencies')] = development_dependencies

                        data.append(row)

                except Exception as e:
                    logging.error(f"Error processing {gem_file_path}: {e}")

    df = pd.DataFrame(data, columns=columns)

    # 각 요소를 검사하여 None, 빈 문자열, 빈 리스트 등을 np.nan으로 통일
    df = df.applymap(lambda x: np.nan if x in [None, '', {}, []] else x)

    # DataFrame을 CSV 파일로 저장
    os.makedirs(output_path, exist_ok=True)
    df.to_csv(os.path.join(output_path, output_filename), index=False)

    logging.info(f"Data extraction complete. CSV file saved at {os.path.join(output_path, output_filename)}")
    
    # 데이터 추출 완료 메시지 출력
    print("Data extraction complete.")
    
    # 상위 5개 행 출력
    print(df.head())
    
    return df

# 함수 실행 예시
df_metadata = extract_package_metadata("../data/package/neutral/")

C:\Users\besis\AppData\Local\Temp\ipykernel_7804\3564904754.py:106: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: np.nan if x in [None, '', {}, []] else x)
2024-10-14 20:42:20,883 - INFO - Data extraction complete. CSV file saved at ../data/metadata/raw\extracted_metadata.csv


Data extraction complete.
Empty DataFrame
Columns: [id, name, version, authors, email, summary, description, date, files, test_files, autorequire, executables, require_paths, dependencies, runtime_dependencies, development_dependencies, extensions, requirements, homepage, metadata, licenses, platform, required_ruby_version, required_rubygems_version, rubygems_version, extra_rdoc_files, rdoc_options, specification_version, cert_chain, signing_key, post_install_message]
Index: []

[0 rows x 31 columns]


In [ ]:
import pandas as pd

def check_duplicates(input_path="../data/metadata/raw/", 
                     input_name="metadata.csv", 
                     subset_columns=['name', 'version', 'rubygems_version'], 
                     output_path="../data/metadata/raw/", 
                     output_name="checked_metadata.csv", 
                     duplicates_output_path="../data/metadata/raw/duplicate_metadata.csv"):
    """
    메타데이터 파일에서 중복된 행을 확인하고 처리한 후 결과를 저장하는 함수.
    
    :param input_path: 데이터 파일이 저장된 입력 경로 (기본값: "../data/metadata/raw/")
    :param input_name: 입력 파일 이름 (기본값: "metadata.csv")
    :param subset_columns: 중복을 확인할 열들의 리스트 (기본값: ['name', 'version', 'rubygems_version'])
    :param output_path: 중복 제거된 데이터를 저장할 경로 (기본값: "../data/metadata/raw/")
    :param output_name: 중복 제거된 데이터를 저장할 파일 이름 (기본값: "checked_metadata.csv")
    :param duplicates_output_path: 중복된 데이터를 저장할 경로 (기본값: "../data/metadata/raw/duplicate_metadata.csv")
    """
    # 데이터 읽기
    df = pd.read_csv(input_path + input_name)

    # 중복된 행을 찾아 출력 및 DataFrame으로 저장
    duplicates = df[df.duplicated(subset=subset_columns, keep=False)]
    if not duplicates.empty:
        print("중복된 행의 정보:")
        print(duplicates[subset_columns])
        
        # 중복된 행을 별도의 파일로 저장
        duplicates.to_csv(duplicates_output_path, index=False)
        print(f"중복된 메타데이터가 {duplicates_output_path}에 저장되었습니다.")
    else:
        print("중복된 행이 없습니다.")

    # 중복된 행을 제거
    df_checked = df.drop_duplicates(subset=subset_columns, keep='first')

    # 중복 제거 전/후 행 개수 확인
    print(f"중복 제거 전 데이터프레임 크기: {df.shape}")
    print(f"중복 제거 후 데이터프레임 크기: {df_checked.shape}")

    # 중복 제거된 데이터를 새로운 파일로 저장
    df_checked.to_csv(output_path + output_name, index=False)
    print(f"중복 제거된 데이터가 {output_path + output_name}에 저장되었습니다.")

# 함수 호출 예시 (기본값으로 실행)
check_duplicates()